<a href="https://colab.research.google.com/github/karishmasd/ML/blob/main/Logistic_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import math
import copy
import matplotlib.pyplot as plt
%matplotlib inline


#**Data**